In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import os 
import glob
import pprint

1. Install ElasticSearch (ES).
2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik

In [2]:
es = Elasticsearch('http://localhost:9200')
es.info()

3. Define an ES analyzer for Polish texts containing:
    - standard tokenizer
    - synonym filter with the following definitions:\
        a. kpk - kodeks postępowania karnego\
        b. kpc - kodeks postępowania cywilnego\
        c. kk - kodeks karny\
        d. kc - kodeks cywilny
    - Morfologik-based lemmatizer
    - lowercase filter
<br><br>
4. Define an ES index for storing the contents of the legislative acts.

In [ ]:
# Old index
es.indices.create(
    index = 'my_index',
    body = {
    "settings": {
        "index": {
            "analysis": {
                "tokenizer": "standard",
                "filter": {
                    "kodeks_synonym_filter": {
                        "type": "synonym",
                        "synonyms": [
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego",
                            "kk => kodeks karny",
                            "kc => kodeks cywilny",
                        ]
                    }
                },
                "analyzer": "morfologik",
            }
        }
    }
})


In [ ]:
# Index with analyzer
es.indices.create(
    index='my_index13',
    body={
        "settings": {
            "index": {
                "analysis": {
                    "filter": {
                        "kodeks_synonym_filter": {
                            "type": "synonym",
                            "synonyms": [
                                "kpk => kodeks postępowania karnego",
                                "kpc => kodeks postępowania cywilnego",
                                "kk => kodeks karny",
                                "kc => kodeks cywilny",
                            ]
                        }
                    },
                    "analyzer": {
                        "default": {
                            "type": "custom",
                            "tokenizer": "standard",
                            "filter": ["kodeks_synonym_filter", "morfologik_stem", "lowercase"]
                        }
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "act": {
                    "type": "text",
                    "term_vector": "with_positions_offsets"
                }
            }
        }
    })

In [21]:
# Test index (check if synonyms and lemmatization work fine)
es.indices.analyze(
    {
        "text": "ustawa, ustawy, ustaw, ustawami, kc kpc itd"
    },
    "my_index13"
)

<ipython-input-21-6bdfabe5227b>:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.analyze(


{'tokens': [{'token': 'ustawa',
   'start_offset': 0,
   'end_offset': 6,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'ustawa',
   'start_offset': 8,
   'end_offset': 14,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'ustawa',
   'start_offset': 16,
   'end_offset': 21,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'ustawić',
   'start_offset': 16,
   'end_offset': 21,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'ustawa',
   'start_offset': 23,
   'end_offset': 31,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'kodeks',
   'start_offset': 33,
   'end_offset': 35,
   'type': 'SYNONYM',
   'position': 4},
  {'token': 'cywilny',
   'start_offset': 36,
   'end_offset': 39,
   'type': 'SYNONYM',
   'position': 5},
  {'token': 'kodeks',
   'start_offset': 36,
   'end_offset': 39,
   'type': 'SYNONYM',
   'position': 5},
  {'token': 'i tak dalej',
   'start_offset': 40,
   'end_offset': 43,
   'type': '<ALPHANUM>',
   'position': 6},
  {'t

5. Load the data to the ES index.

In [ ]:
# For those without mapping [?]
for file in glob.glob('../ustawy/*'):
    fname = os.path.basename(file)
    with open(file, encoding='UTF-8') as f:
        text = f.read()
        es.create(
            index="my_index13",
            id=fname,
            body={"text": text},
            doc_type="act"
        )

In [60]:
for file in glob.glob('../ustawy/*'):
    fname = os.path.basename(file)
    with open(file, encoding='UTF-8') as f:
        text = f.read()
        es.create(
            index="my_index13",
            id=fname,
            body={"text": text},
        )

<ipython-input-60-12e424add7ce>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.create(


6. Determine the number of legislative acts containing the word ustawa (in any form).

In [22]:
containing_ustawa = es.search(
    body={
        "query": {
            "match": {
                "text": {
                    "query": "ustawa"
                }
            }
        }
    },
    index="my_index10",
)
containing_ustawa["hits"]["total"]["value"]

<ipython-input-22-e7cfe6d0d02c>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  containing_ustawa = es.search(


1178

7. Determine the number of occurrences of the word ustawa by searching for this particular form, including the other inflectional forms.

In [7]:
# Preapre IDs array to search every file (not needed for termvectors).
IDs = []
for file in glob.glob('../ustawy/*'):
    fname = os.path.basename(file)
    IDs.append(fname)

In [8]:
# zrun termvectors once. Then we can get needed inforamtion from results.
ID = '1993_645.txt'
vector = es.termvectors(
    index="my_index13",
    id=ID,
    fields="text",
    term_statistics=True
)

In [23]:
try:
    print(vector['term_vectors']['text']['terms']['ustawa']['ttf'])
except KeyError :
    print("Word not found.")

24934


8. Determine the number of occurrences of the word ustaw by searching for this particular form, including the other inflectional forms.

In [24]:
try:
    print(vector['term_vectors']['text']['terms']['ustaw']['ttf'])
except KeyError :
    print("Word not found.")


Word not found.


In [ ]:
# Search query, but searches only for files.
es.search(
    index="my_index13",
    body={
        "query": {
            "match": {
                "text": {
                    "query": "ustawa",
                }
            }
        },
    }
)

Can't search for words.

In [ ]:
es.search(
    index="my_index13",
    query={
        "query_string": {
            "query": "ustaw"
        }
    }
)

9.  Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in any inflection form.

In [28]:
es.search(
    index="my_index13",
    body={
        "query": {
            "match_phrase": {
                "text": {
                    "query": "kodeks postępowania cywilnego"
                }
            }
        }
    }
)["hits"]["total"]["value"]

<ipython-input-28-3c3b49924486>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


99

10. Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.


In [15]:
es.search(
    index="my_index13",
    body={
        "query": {
            "match_phrase": {
                "text": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }
            }
        }
    }
)["hits"]["total"]["value"]

<ipython-input-15-47b31e66faae>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


1174

11. Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [29]:
konstytucja = es.search(
    index="my_index13",
    body={
        "query": {
            "match": {
                "text": {
                    "query": "konstytucja",
                }
            }
        },
        "highlight": {
            "fields": {
                "text": {}
            },
            "number_of_fragments": 3
        }
    },
    sort="_score",
    size="10"
)["hits"]["hits"]

best_scores = [d['_score'] for d in konstytucja]
best_ids = [d['_id'] for d in konstytucja]
best_ids, best_scores

<ipython-input-29-d5186e05ad64>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  konstytucja = es.search(


(['1997_629.txt',
  '2000_443.txt',
  '1997_604.txt',
  '1996_350.txt',
  '1997_642.txt',
  '2001_23.txt',
  '1996_199.txt',
  '1999_688.txt',
  '1997_681.txt',
  '2001_1082.txt'],
 [6.869184,
  6.663479,
  6.632288,
  6.6273947,
  6.2522817,
  6.056855,
  5.9267144,
  5.848894,
  5.4653444,
  5.4653444])

12. Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.

In [30]:
best_highlight = [d['highlight'] for d in konstytucja]
pprint.pprint(best_highlight)

[{'text': ['o zmianie ustawy konstytucyjnej o trybie przygotowania\n'
           '           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej',
           'W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie '
           'przygotowania i \n'
           'uchwalenia <em>Konstytucji</em>',
           'Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz \n'
           '                obywateli popierających zgłoszenie']},
 {'text': ['umowy międzynarodowej i nie wypełnia przesłanek określonych w art. '
           '89\n'
           '     ust. 1 lub art. 90 <em>Konstytucji</em>',
           'międzynarodowej lub załącznika nie\n'
           '     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 '
           '<em>Konstytucji</em>',
           'co do zasadności wyboru\n'
           '  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 '
           'ust. 2\n'
           '  <em>Konstytucji</em>']},
 {'text': ['Jeżeli Trybunał Konstytucyjny wyda orz